In [1]:
import pandas as pd
import os

print("🚀 STARTING DATA REFINERY...")

# --- PART 1: LOAD RAW DATA (Robust Mode) ---
# We use engine='python' to avoid ParserErrors with special characters
try:
    print("1. Loading Raw Data...")
    df_orders = pd.read_csv('../data/raw_orders.csv', encoding='latin1', sep=None, engine='python')
    df_returns = pd.read_csv('../data/raw_returns.csv', encoding='latin1', sep=None, engine='python')
    df_people = pd.read_csv('../data/raw_people.csv', encoding='latin1', sep=None, engine='python')
    print(f"   ✅ Loaded {len(df_orders)} orders.")
except FileNotFoundError:
    print("   ❌ ERROR: Raw CSVs not found. Please ensure 'raw_orders.csv' exists in 'data/'.")
    raise

# --- PART 2: CLEANING THE MAIN TABLE ---
print("2. Cleaning Orders Data...")
df_orders_clean = df_orders.copy()

# A. Fix Postal Code (Missing & Type)
# Fill NaN with '00000' and ensure it's a string, not a float (remove '.0')
df_orders_clean['Postal Code'] = df_orders_clean['Postal Code'].fillna('00000')
df_orders_clean['Postal Code'] = df_orders_clean['Postal Code'].astype(str).str.replace('.0', '', regex=False)

# B. Fix Dates (Critical for SQL)
# Convert to datetime objects first, then force to YYYY-MM-DD string format for SQLite
print("   - Standardizing dates to YYYY-MM-DD...")
df_orders_clean['Order Date'] = pd.to_datetime(df_orders_clean['Order Date'], dayfirst=True, errors='coerce').dt.strftime('%Y-%m-%d')
df_orders_clean['Ship Date'] = pd.to_datetime(df_orders_clean['Ship Date'], dayfirst=True, errors='coerce').dt.strftime('%Y-%m-%d')

# Drop rows where dates failed to parse (bad data integrity)
df_orders_clean = df_orders_clean.dropna(subset=['Order Date', 'Ship Date'])

# C. Fix Region (Feature Engineering)
# Create 'Market_Group' to separate US vs Global
us_regions = ['East', 'West', 'Central', 'South']
df_orders_clean['Market_Group'] = df_orders_clean['Region'].apply(lambda x: 'USCA' if x in us_regions else 'Global')

# --- PART 3: PREPARE LOOKUP TABLES ---
df_returns_clean = df_returns.copy()
df_people_clean = df_people.copy()

# --- PART 4: SAVE TO DISK (The Handoff) ---
print("3. Saving Processed Data...")
processed_path = '../data/processed'
os.makedirs(processed_path, exist_ok=True)

df_orders_clean.to_csv(f'{processed_path}/orders_clean.csv', index=False)
df_returns_clean.to_csv(f'{processed_path}/returns_clean.csv', index=False)
df_people_clean.to_csv(f'{processed_path}/people_clean.csv', index=False)

print(f"✅ SUCCESS: Clean data saved to {os.path.abspath(processed_path)}")

🚀 STARTING DATA REFINERY...
1. Loading Raw Data...
   ✅ Loaded 51290 orders.
2. Cleaning Orders Data...
   - Standardizing dates to YYYY-MM-DD...
3. Saving Processed Data...
✅ SUCCESS: Clean data saved to c:\Projects\p2-global-sales-dashboard\data\processed
